In [8]:
# imports

import face_recognition as fr
import os
import cv2
from gcloud import storage
import io
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
import audioread
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM # some texts use GMM
from sklearn import preprocessing # break the blackbox here
import glob

In [4]:
# constant parameters

jsonPath = '/home/subbu/PRML/Internship/p1/rankingtranscript-96316cee08ba.json'
bucketName = 'rankingtranscript1'

In [5]:
def photo_face_recognition(input_photo,testing_pic):
    
    # downloading files from cloud
    storage_client = storage.Client.from_service_account_json(jsonPath)
    bucket = storage_client.get_bucket(bucketName)
    blob = bucket.get_blob(input_photo)
    blob.download_to_filename(input_photo)
    blob2 = bucket.get_blob(testing_pic)
    blob2.download_to_filename(testing_pic)
    
    picture = fr.load_image_file(input_photo)
    face_encoding = fr.face_encodings(picture)[0]


    uk_picture = fr.load_image_file(testing_pic)
    uk_encoding = fr.face_encodings(uk_picture)[0]


    results = fr.compare_faces([face_encoding], uk_encoding)
    
    os.remove(input_photo)
    os.remove(testing_pic)
    
    count = 0
    for output in results:
        if(output == True):
            count += 1
    if(count > 0):
        return True
    else:
        return False


In [6]:
def video_face_recognition(input_photo,testing_video):
    
    # downloading files from cloud
    storage_client = storage.Client.from_service_account_json(jsonPath)
    bucket = storage_client.get_bucket(bucketName)
    blob = bucket.get_blob(input_photo)
    blob.download_to_filename(input_photo)
    blob2 = bucket.get_blob(testing_video)
    blob2.download_to_filename(testing_video)
    
    pic = fr.load_image_file(input_photo)
    input_face_encoding = fr.face_encodings(pic)[0]
    
    vid = cv2.VideoCapture(testing_video)
    frms = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    print(frms)
    if(frms > 150):
        num = 100
    else:
        num = frms - 60
        
    frame_cnt = 0
    while(frame_cnt < num + 50):
        ret,frame = vid.read() 
        if ret:
            name = 'frame' + str(frame_cnt) + '.jpg'
            if(frame_cnt >= num):
                cv2.imwrite(name, frame)
            frame_cnt += 1
        else: 
            break
    print('created images '+str(frame_cnt))
    
    count = num
    match = 0
    while(count < num + 50):
        pic = fr.load_image_file('frame' + str(count) + '.jpg')
        try:
            encode = fr.face_encodings(pic)[0]
        except:
            count += 1
            continue
        results = fr.compare_faces([input_face_encoding], encode)
        count += 1
        if(results[0]):
            match += 1
    
    print(match)
    
    # deleting local files
    
    frame_cnt = 0
    while(frame_cnt < num + 50):
        name = 'frame' + str(frame_cnt) + '.jpg'
        if(frame_cnt >= num):
            os.remove(name)
        frame_cnt += 1
    print('deleted images '+str(frame_cnt))
    
    os.remove(input_photo)
    os.remove(testing_video)
    
    if(match > 0):
        return True
    else:
        return False

In [12]:
def audio_recognition(input_audio,test_directory):
    
    sr,audio = read(input_audio)
    
    feature = mfcc.mfcc(audio,sr, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)
    feature = preprocessing.scale(feature)
    
    gmm = GMM(n_components = 16, max_iter = 200, covariance_type='diag',n_init = 3)
    gmm.fit(feature)
    
    name = ''
    score = -100000
    wav_files = glob.glob(test_directory+'/*')
    for one in wav_files:
        srx,audiox = read(one)
        featurex = mfcc.mfcc(audiox,srx, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)
        featurex = preprocessing.scale(featurex)
        scorex = gmm.score(featurex)
        if(score < scorex):
            score = scorex
            name = one
    name = name[name.index('/')+1:name.index('.')]
    
    return name


In [ ]:
# trail runs

In [7]:
video_face_recognition("obama.jpg","test_obama.mp4")

1737.0
created images 150
50
deleted images 150


True

In [13]:
audio_recognition('SampleData/Karan_17.wav','SampleData')

'Karan_17'